# Import Library

In [ ]:
import pandas as pd
import torch
from os.path import exists
from facenet_pytorch import MTCNN, extract_face
from PIL import Image, ImageDraw

# Import CSV files and Prepross

In [ ]:
labeled_users = './data/labeled_users.csv'
user_demo_profiles_path = './data/User demo profiles.json'

df_users = pd.read_csv(labeled_users)
df_profiles = pd.read_json(user_demo_profiles_path, orient='values')

In [ ]:
df_users = df_users.drop(['is_female','year_born'],axis = 1)
df_users = df_users.rename(columns={"user_id":"id","race":"race"})
df_users.head()

In [ ]:
df_profiles = df_profiles.drop(['name','screen_name','description','lang'],axis = 1)
df_profiles.head()

In [ ]:
data = df_users.join(df_profiles.set_index('id'), on='id')
data = data.dropna()
data['img_path'] = './data/' + data['img_path']
data.head()

In [ ]:
print(data.shape)
to_drop = []
for i, row in enumerate(data.iterrows()):
    index, row = row
    path = row['img_path']
    if not exists(path):
        to_drop.append(index)
data = data.drop(to_drop)
print(data.shape)

# Import a face detection model

In [ ]:
# little test
test_path = './data/profile pics/60152.jpeg'#data['img_path'][0]
img = Image.open(test_path)
img.show()

In [ ]:
# face detector
mtcnn = MTCNN(keep_all=True,select_largest=True)
boxes, probs, points = mtcnn.detect(img, landmarks=True)

In [ ]:
# Draw boxes and save faces
extract = extract_face(img, boxes[0], save_path='detected_face_{}.png'.format(i))

# Get All faces

In [ ]:
def get_all_faces(input):
    return Image.open(input)

In [ ]:
data['img'] = data['img_path'].apply(get_all_faces)

In [ ]:
boxes, probs = mtcnn.detect(data['img'].to_list())

In [ ]:
data['boxes'] = boxes

In [ ]:
def get_extract(para):
    image, boxes = para[0], para[1]
    if boxes is None:
        return pd.NA
    else:
        return extract_face(image, boxes[0])

In [ ]:
data['tensor'] = data[['img','boxes']].apply(get_extract, axis = 1)

In [ ]:
data = data.drop(columns=['img_path','img','boxes'])
data = data.dropna()
data.head()

In [ ]:
data.to_pickle('./Dataset1_id_Race_FaceTensor.pkl')